#Simple house modeling with random texturized floors

##Changes
* the main function can manage multi storey house, building storey one by one
* the main function can manage the insertion of doors and windows in the spatial frame
* two specialized functions were created to manage the insertion, one for the doors and one for the windows

##Helper and main functions (some \n inserted for the sake of readability)

```python
def generate_windows_special_hole_models(linesFileName, modelBuilder, height):
	"""
	generate_windows_special_hole_models is a function that given a .lines's file name, 
    a model builder function and a wall height, generate a couple containing two lists: 
    the former contain the windows models, the latter contain the windows model used to 
    generate the holes in the walls.
	@param linesFileName: the file name of .lines file containing lines used to place windows models
	@param modelBuilder: the windows's generator function, accept the window 3 dimensions
	@param height: the wall height used to position the models
	@return (models, holes): models contain the actual models of the windows meanwhile holes contain
	the convex hulls of the corresponding element in models list
	"""
	with open("lines/"+ linesFileName + ".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		holes = []
		models = []
		xMin = 100000000
		xMax = -100000000
		yMin = 100000000
		yMax = -100000000
		for row in reader:
			xMin = min(xMin, float(row[0]), float(row[2]))
			xMax = max(xMax, float(row[0]), float(row[2]))
			yMin = min(yMin, float(row[1]), float(row[3]))
			yMax = max(yMax, float(row[1]), float(row[3]))
			if float(row[0]) != float(row[2]):
				model = modelBuilder(xMax-xMin,19.5,height/2.)
				model = T([1,2,3])([xMin,yMin*.99, height/4.])(model)
			else:
				model = modelBuilder(yMax-yMin,19.5,height/2.)
				model = R([1,2])(-PI/2.)(model)
				model = T([2])([yMax-yMin])(model)
				model = T([1,2,3])([xMin*.99,yMin, height/4.])(model)
			holes.append(JOIN(SKEL_1(model)))
			models.append(model)
			xMin = 100000000
			xMax = -100000000
			yMin = 100000000
			yMax = -100000000
	holes = STRUCT(holes)
	models = STRUCT(models)
	return (models, holes)

def generate_doors_special_hole_models(linesFileName, modelBuilder, height):
	"""
	generate_doors_special_hole_models is a function that given a .lines's file name, 
    a model builder function and a wall height, generate a couple containing two lists: 
    the former contain the doors models, the latter	contain the doors model used to 
    generate the holes in the walls.
	@param linesFileName: the file name of .lines file containing lines used to place doors models
	@param modelBuilder: the doors's generator function, accept the door 3 dimensions 
	@param height: the wall height used to position the models
	@return (models, holes): models contain the actual models of the windows meanwhile, holes contain
	the convex hulls of the corresponding element in models list
	"""
	with open("lines/"+ linesFileName + ".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		holes = []
		models = []
		xMin = 100000000
		xMax = -100000000
		yMin = 100000000
		yMax = -100000000
		for row in reader:
			xMin = min(xMin, float(row[0]), float(row[2]))
			xMax = max(xMax, float(row[0]), float(row[2]))
			yMin = min(yMin, float(row[1]), float(row[3]))
			yMax = max(yMax, float(row[1]), float(row[3]))
			if float(row[0]) != float(row[2]):
				model = modelBuilder(xMax-xMin,19,3*height/4.)
				model = T([1,2])([xMin,yMin*.99])(model)
			else:
				model = modelBuilder(yMax-yMin,19,3*height/4.)
				model = R([1,2])(-PI/2.)(model)
				model = T([2])([yMax-yMin])(model)
				model = T([1,2])([xMin*.99,yMin])(model)
			holes.append(JOIN(SKEL_1(model)))
			models.append(model)
			xMin = 100000000
			xMax = -100000000
			yMin = 100000000
			yMax = -100000000
	holes = STRUCT(holes)
	models = STRUCT(models)
	return (models, holes)
    
def texturized_floors(story, ladderHoleModel = False):
	"""
	texturized_floors is a function that return a list of HPC models, 
    in particular models of the different floors that are present in the building, 
    including the external floors. two params are accepted, the current story generated and
	eventually the ladder model used to generate hole in the floors, 
    however this function build all the floors of 4 type of environments: 
    livingroom, bathroom, bedroom, terrace.
	Moreover, in order to generate correctly all the floors, the following files are needed: 
    bedroomKX.lines, where X is an integer > 0, bathroomKY.lines where Y is an integer > 0, 
    livingroomKZ.lines where Z in an integer > 0, terrace_surfaceKW.lines where W is an integer > 0. 
    It actually possible to have a series of file (e.g. bathroom1.lines, bathroom2.lines, ecc...), 
    K is the number corresponding to the current story. 
	In addition, this function could add some fancy random texture to the generated 
    floors (up to 6 for every category)
	@param story: current story
	@param ladderHoleModel: the ladder model used to calculate the holes in the floor 
	@return res: list of HPCs representing all the floors generated with their texture if any
	"""
	res = []
	def build_floor(roomType, texturePrefix):
		counter = 1
		result = []
		while True:
			if os.path.isfile("lines/" + roomType + str(story) + str(counter) + ".lines"):
				with open("lines/" + roomType + str(story) + str(counter) + ".lines", "rb") as file:
					reader = csv.reader(file, delimiter=",")
					polylineList = []
					for row in reader:
						polylineList.append(POLYLINE([[float(row[0]),float(row[1])],[float(row[2]), float(row[3])]]))
				floor = PROD([SOLIDIFY(STRUCT(polylineList)),Q(.5)])
				if(story != 0 and ladderHoleModel):
                    ladderHole = T([3])([-1])(ladderHoleModel)
					floor = DIFFERENCE([floor, ladderHole])
				result.append(TEXTURE("texture/" + texturePrefix+str(randint(1,6))+".jpg")(floor))
				counter = counter + 1
			else: 
				counter = 1
				break
		return result
	res = res + build_floor("bedroom","camera")
	res = res + build_floor("bathroom", "bagno")
	res = res + build_floor("livingroom", "sala")
	res = res + build_floor("terrace_surface", "terrazzo")
	return res
```

###Main function
```python
def build_house(story, 
    windowsGenerationFunction = False, 
    doorsGenerationFunction = False, 
    ladderHoleModel = False):
	"""
	build_house is the main function that given a story identifier build the spatial frame (barebone)
	of the story, optionally it takes ulterior three optional parameter, 
    representing a window generating function, a door generating function and a ladder hole model; 
    the firsts two are used to insert windows and doors inside the spatial frame, 
    meanwhile the last one is used to generate the hole	in the floors, 
    in order to make space for the spiral stair.
	@param story: the identifier of the current story (e.g. 0 ground floor, 1 first floor, ecc...)
	@param windowsGenerationFunction: the windows generation function, accept window's dimension (optional)
	@param doorsGenerationFunction: the doors generation function, accept door's dimension (optional)
	@param ladderHoleModel: the hole's model of the stair (optional)
	"""
    
	#generating 2D external walls
	externalWalls = generate_2D_walls("muriesterni"+str(story))

	#defining scaling factors
	xfactor = 15/SIZE([1])(externalWalls)[0]
	yfactor = 15.1/SIZE([2])(externalWalls)[0]
	zfactor = xfactor

	#building external 3D-walls
	walls = OFFSET([12,12])(externalWalls)
	walls = PROD([walls, Q(3/xfactor)])

	#generating internal 2D-walls
	internalWalls = generate_2D_walls("muriinterni"+str(story))

	#building internal 3D-walls
	internals = OFFSET([7,7])(internalWalls)
	internals = PROD([internals, Q(3/xfactor)])

	#building 3D-doors holes
	if(not doorsGenerationFunction):
		doors = generate_hole_models("porte"+str(story))
		doors = PROD([doors, Q(2.5/xfactor)])
	else:
		doors = generate_doors_special_hole_models(
            "porte_model"+str(story),
            doorsGenerationFunction,
            3/zfactor)

	#building 3D-windows holes
	if(not windowsGenerationFunction):
		windows = generate_hole_models("finestre"+str(story))
		windows = PROD([windows, Q(SIZE([3])(walls)[0]/2.)])
		windows = T(3)(SIZE([3])(walls)[0]/4.)(windows)
	else:
		windows = generate_windows_special_hole_models(
            "finestre_model"+str(story),
            windowsGenerationFunction,
            4/zfactor)

	#building 2D-model of the terrace walls
	terrace_walls = generate_2D_walls("terrace_walls"+str(story))
	terrace_walls = OFFSET([5,5])(terrace_walls)

	#building 3D-model of the terrace walls with some fancy texture addition
	terrace_walls = PROD([terrace_walls, Q(1.5/xfactor)])
	terrace_walls = TEXTURE(["texture/wood2.jpg",True,True,10,10,PI/2.,200,200,100,100])(terrace_walls)

	#building the frame assembling the external walls and the interior walls
	frame = STRUCT([walls, internals])

	#breaking the walls in order to put some windows and doors
	if (not windowsGenerationFunction and not doorsGenerationFunction):
		exteriors = DIFFERENCE([walls, windows, doors])
		interiors = DIFFERENCE([internals, doors, windows])
	if (windowsGenerationFunction and not doorsGenerationFunction): 
		exteriors = DIFFERENCE([walls, doors, windows[1]])
		interiors = DIFFERENCE([internals, doors, windows[1]])
	if (not windowsGenerationFunction and doorsGenerationFunction):
		exteriors = DIFFERENCE([walls, doors[1], windows])
		interiors = DIFFERENCE([internals, doors[1], windows])
	if (windowsGenerationFunction and doorsGenerationFunction):
		exteriors = DIFFERENCE([walls, doors[1], windows[1]])
		interiors = DIFFERENCE([internals, doors[1], windows[1]])
		

	#adding some fancy texture to the walls
	exteriors = TEXTURE(["texture/wood_surface.jpg",True,True,10,10,-PI/2.,1,1])(exteriors)
	interiors = TEXTURE(["texture/wood1.jpg",True,True,1,1,PI/2.,5,5])(interiors)

	#building the floors
	floor = STRUCT(texturized_floors(story, ladderHoleModel))

	#scaling and assembling all together
	if (not windowsGenerationFunction and not doorsGenerationFunction):	
		house = S([1,2,3])
            ([xfactor,yfactor, zfactor])
            (STRUCT([interiors, exteriors, terrace_walls, floor]))
	if (windowsGenerationFunction and not doorsGenerationFunction):
		house = S([1,2,3])
            ([xfactor,yfactor, zfactor])
            (STRUCT([interiors, exteriors, terrace_walls, floor, windows[0]]))
	if (not windowsGenerationFunction and doorsGenerationFunction):
		house = S([1,2,3])
        ([xfactor,yfactor, zfactor])
        (STRUCT([interiors, exteriors, terrace_walls, floor, doors[0]]))
	if (windowsGenerationFunction and doorsGenerationFunction):
		house = S([1,2,3])
        ([xfactor,yfactor, zfactor])
        (STRUCT([interiors, exteriors, terrace_walls, floor, windows[0], doors[0]]))
        
	return house
```

##Results (the actual parameters are the same used in workshop_10)

<table>
 <tbody>
    <tr>
        <td><img src="../img/casa12.png" style="width: 400px; height: 300px"></td>
        <td><img src="../img/casa13.png" style="width: 400px; height: 300px"></td>
    </tr>
</tbody>
</table>